# Predicting Drug_type in Train

### Notebook automatically generated from your model

Model Random forest (s1), trained on 2022-10-19 04:09:53.

#### Generated on 2022-10-20 04:18:28.733510

prediction
This notebook will reproduce the steps for a MULTICLASS on  Train.
The main objective is to predict the variable Drug_type

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [2]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [3]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'bigint', u'name': u'Age'}, {u'type': u'bigint', u'name': u'Gender'}, {u'type': u'bigint', u'name': u'BP_HIGH'}, {u'type': u'bigint', u'name': u'BP_NORMAL'}, {u'type': u'bigint', u'name': u'BP_LOW'}, {u'type': u'bigint', u'name': u'Cholesterol'}, {u'type': u'double', u'name': u'Sodium and potassium Level'}, {u'type': u'string', u'name': u'Drug_type'}]}

ml_dataset_handle = dataiku.Dataset('Train')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 9.11 ms, sys: 2.08 ms, total: 11.2 ms
Wall time: 39 ms
Base data has 140 rows and 8 columns


,Age,Gender,BP_HIGH,BP_NORMAL,BP_LOW,Cholesterol,Sodium and potassium Level,Drug_type
0,23,0,1,0,0,1,25.355,DrugY
1,47,1,0,0,1,1,13.093,drugC
2,47,1,0,0,1,1,10.114,drugC
3,60,1,0,1,0,1,15.171,DrugY
4,43,1,0,0,1,0,19.368,DrugY


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [4]:
ml_dataset = ml_dataset[[u'Drug_type', u'BP_NORMAL', u'Age', u'BP_LOW', u'Gender', u'Cholesterol', u'Sodium and potassium Level', u'BP_HIGH']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [5]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = []
numerical_features = [u'BP_NORMAL', u'Age', u'BP_LOW', u'Gender', u'Cholesterol', u'Sodium and potassium Level', u'BP_HIGH']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We are now going to handle the target variable and store it in a new variable:

In [6]:
target_map = {u'drugC': 4, u'drugB': 3, u'DrugY': 0, u'drugX': 1, u'drugA': 2}
ml_dataset['__target__'] = ml_dataset['Drug_type'].map(str).map(target_map)
del ml_dataset['Drug_type']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

ml_dataset['__target__'] = ml_dataset['__target__'].astype(np.int64)

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [7]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

Train data has 112 rows and 8 columns
Test data has 28 rows and 8 columns


#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [8]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u'BP_NORMAL'}, {'impute_with': u'MEAN', 'feature': u'Age'}, {'impute_with': u'MEAN', 'feature': u'BP_LOW'}, {'impute_with': u'MEAN', 'feature': u'Gender'}, {'impute_with': u'MEAN', 'feature': u'Cholesterol'}, {'impute_with': u'MEAN', 'feature': u'Sodium and potassium Level'}, {'impute_with': u'MEAN', 'feature': u'BP_HIGH'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

/home/ruser1/dataiku-dss-11.0.0/python.packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Imputed missing values in feature BP_NORMAL with value 0.276785714286
Imputed missing values in feature Age with value 43.9821428571
Imputed missing values in feature BP_LOW with value 0.339285714286
Imputed missing values in feature Gender with value 0.5625
Imputed missing values in feature Cholesterol with value 0.535714285714
Imputed missing values in feature Sodium and potassium Level with value 15.7244375
Imputed missing values in feature BP_HIGH with value 0.383928571429


We can now handle the categorical features (still using the settings defined in Models):

Let's rescale numerical features

In [9]:
rescale_features = {u'BP_NORMAL': u'AVGSTD', u'Age': u'AVGSTD', u'BP_LOW': u'AVGSTD', u'Gender': u'AVGSTD', u'Cholesterol': u'AVGSTD', u'Sodium and potassium Level': u'AVGSTD', u'BP_HIGH': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

Rescaled BP_NORMAL
Rescaled Age
Rescaled BP_LOW
Rescaled Gender
Rescaled Cholesterol


/home/ruser1/dataiku-dss-11.0.0/python.packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Rescaled Sodium and potassium Level
Rescaled BP_HIGH


#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [10]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,
    random_state=1337,
    max_depth=6,
    min_samples_leaf=1,
    verbose=2)

We set "class_weight" as the weighting strategy:

In [12]:
clf.class_weight = "balanced"

... And train the model

In [13]:
%time clf.fit(X_train, y_train)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=1337, verbose=2, warm_start=False)

Build up our result dataset

The model is now trained, we can apply it to our test set:

In [14]:
%time _predictions = clf.predict(X_test)
%time _probas = clf.predict_proba(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]
probabilities = pd.DataFrame(data=_probas, index=X_test.index, columns=cols)

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(probabilities, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'Drug_type'})

CPU times: user 7.57 ms, sys: 2.28 ms, total: 9.85 ms
Wall time: 9.09 ms
CPU times: user 6.32 ms, sys: 2.15 ms, total: 8.47 ms
Wall time: 7.93 ms


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Let's have a look at feature importances

In [15]:
feature_importances_data = []
features = X_train.columns
for feature_name, feature_importance in zip(features, clf.feature_importances_):
    feature_importances_data.append({
        'feature': feature_name,
        'importance': feature_importance
    })

# Plot the results
pd.DataFrame(feature_importances_data)\
    .set_index('feature')\
    .sort_values(by='importance')[-10::]\
    .plot(title='Top 10 most important variables',
          kind='barh',
          figsize=(10, 6),
          color='#348ABD',
          alpha=0.6,
          lw='1',
          edgecolor='#348ABD',
          grid=False,)

#### Results

You can measure the model's accuracy:

In [16]:
from dataiku.doctor.utils.metrics import mroc_auc_score
y_test_ser = pd.Series(y_test)
 
print ('AUC value:', mroc_auc_score(y_test_ser, _probas))

('AUC value:', 1.0)


We can also view the predictions directly.
Since scikit-learn only predicts numericals, the labels have been mapped to 0,1,2 ...
We need to 'reverse' the mapping to display the initial labels.

In [17]:
inv_map = { target_map[label] : label for label in target_map}
predictions.map(inv_map)

3      DrugY
10     DrugY
13     DrugY
14     DrugY
20     drugX
22     drugA
25     DrugY
29     DrugY
32     drugB
34     drugB
35     DrugY
42     DrugY
49     DrugY
60     drugX
63     DrugY
68     DrugY
69     DrugY
77     DrugY
83     DrugY
91     DrugY
120    DrugY
122    drugA
123    DrugY
125    DrugY
130    drugB
136    DrugY
137    drugC
139    drugX
Name: predicted_value, dtype: object

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
